In [1]:
using Pkg; Pkg.DEFAULT_IO[] = stdout; Pkg.activate("."); Pkg.instantiate();

  Activating environment at `~/Research/ToyModel.jl/Project.toml`


In [2]:
Pkg.status()

      Status `~/Research/ToyModel.jl/Project.toml`
  [6e4b80f9] BenchmarkTools v1.1.0
  [6fe1bfb0] OffsetArrays v1.10.2
  [91a5bcdd] Plots v1.16.7


In [3]:
using OffsetArrays

L = 2.0
n = 15
dx = L/n

# number of halo points
nh = 2

# a grid with periodic BC
xᶠ = OffsetArray(-nh*dx:dx:L-dx+nh*dx, -nh)
xᶜ = OffsetArray(-nh*dx+dx/2:dx:L-dx/2+nh*dx, -nh);

In [4]:
u₀(x) = sin(2π/L * x)
h₀(x) = cos(4π/L * x);

In [5]:
udata = u₀.(xᶠ)
hdata = h₀.(xᶜ);

In [6]:
∂udata_theoretical =   2π/L * cos.(2π/L * xᶜ)
∂hdata_theoretical = - 4π/L * sin.(4π/L * xᶠ);

In [7]:
udata

19-element OffsetArray(::Vector{Float64}, -1:17) with eltype Float64 with indices -1:17:
 -0.7431448254773941
 -0.40673664307580015
  0.0
  0.40673664307580015
  0.7431448254773941
  0.9510565162951535
  0.9945218953682734
  0.8660254037844387
  0.5877852522924732
  0.20791169081775931
 -0.20791169081775907
 -0.587785252292473
 -0.8660254037844385
 -0.9945218953682733
 -0.9510565162951536
 -0.743144825477394
 -0.40673664307580015
 -2.4492935982947064e-16
  0.4067366430757997

In [8]:
struct Grid
    L
    n
    dx
    xᶠ
    xᶜ
end

grid = Grid(L, n, dx, xᶠ, xᶜ)

Grid(2.0, 15, 0.13333333333333333, -0.26666666666666666:0.13333333333333333:2.1333333333333333 with indices -1:17, -0.2:0.13333333333333333:2.2 with indices -1:17)

In [9]:
faces(grid::Grid) = xᶠ[1:grid.n]
cells(grid::Grid) = xᶜ[1:grid.n];

In [10]:
struct BasicField
    data::AbstractArray
    location::AbstractArray
end

ufield = BasicField(udata, xᶠ)

hfield = BasicField(hdata, xᶜ)

BasicField([0.30901699437494745, 0.9135454576426009, 0.9135454576426009, 0.30901699437494745, -0.4999999999999998, -0.9781476007338057, -0.8090169943749475, -0.10452846326765423, 0.6691306063588585, 1.0, 0.6691306063588589, -0.10452846326765287, -0.8090169943749472, -0.9781476007338055, -0.49999999999999983, 0.309016994374947, 0.9135454576426004, 0.9135454576426009, 0.30901699437494623], -0.2:0.13333333333333333:2.2 with indices -1:17)

In [11]:
using Plots

In [ ]:
plot(ufield.location, ufield.data, marker=:circle)
plot!(hfield.location, hfield.data, marker=:square)

Btw, we can add a method for `Plots.plot()` :)

In [ ]:
import Plots: plot, plot!
Plots.plot(f::BasicField, args...; kwargs...) = plot(f.location, f.data, args...; kwargs...)
Plots.plot!(f::BasicField, args...; kwargs...) = plot!(f.location, f.data, args...; kwargs...)

In [ ]:
plot(ufield, marker=:circle)
plot!(hfield, marker=:square)

Let's make a better `Field` type that contains the location of the field as parameter.

In [ ]:
abstract type AbstractLocation end

In [ ]:
struct Cell <: AbstractLocation end 
struct Face <: AbstractLocation end 

In [ ]:
struct Field{L<:AbstractLocation, D<:AbstractArray, G}
    data :: D
    grid :: G
    
    Field{L}(data::D, grid::G) where {L, D, G} = new{L, D, G}(data, grid)
end

In [ ]:
import Plots: plot, plot!
Plots.plot(f::Field{Face}, args...; kwargs...) = plot(f.grid.xᶠ, f.data, args...; kwargs...)
Plots.plot(f::Field{Cell}, args...; kwargs...) = plot(f.grid.xᶜ, f.data, args...; kwargs...)
Plots.plot!(f::Field{Face}, args...; kwargs...) = plot!(f.grid.xᶠ, f.data, args...; kwargs...)
Plots.plot!(f::Field{Cell}, args...; kwargs...) = plot!(f.grid.xᶜ, f.data, args...; kwargs...)

In [ ]:
u = Field{Face}(udata, grid)
h = Field{Cell}(hdata, grid)

Now the types of the fields include information on whether the field lives on cell centers or interfaces. Thus we can write different function methods based on field type.

In [ ]:
typeof(u)

In [ ]:
typeof(h)

Now let's compute derivatives of fields.

In [ ]:
δᶜ(i, u) = u[i+1] - u[i]

In [ ]:
δᶠ(i, h) = h[i] - h[i-1]

In [ ]:
δ(i, ψ::Field{<:Cell}) = δᶠ(i, ψ.data)
δ(i, ψ::Field{<:Face}) = δᶜ(i, ψ.data);

In [ ]:
δ(3, u)

In [ ]:
δ(3, h)

In [ ]:
∂(i, ψ::Field{<:Cell}) = δᶠ(i, ψ.data) / ψ.grid.dx
∂(i, ψ::Field{<:Face}) = δᶜ(i, ψ.data) / ψ.grid.dx;

In [ ]:
function ∂_arrays(i, ψ::AbstractArray, grid; location="face")
    if location == "face"
        return δᶜ(i, ψ) / grid.dx
    else
        return δᶠ(i, ψ) / grid.dx
    end
    
end

In [ ]:
∂udata = similar(udata)

for i in 1:n
    ∂udata[i] = ∂(i, u)
end

In [ ]:
plot(xᶜ[1:n], ∂udata[1:n])
plot!(xᶜ[1:n], ∂udata_theoretical[1:n])

In [ ]:
using BenchmarkTools

In [ ]:
@btime ∂(10, h);

In [ ]:
@btime ∂_arrays(10, udata, grid; location="cell");

In [ ]:
∂u = Field{Cell}(similar(u.data), grid)
∂h = Field{Face}(similar(h.data), grid)

In [ ]:
function ∂!(∂f::Field, f::Field)
    for i in 1:f.grid.n
        ∂f.data[i] = ∂(i, f)
    end
end;

In [ ]:
@btime ∂!(∂u, u)

In [ ]:
@btime ∂!(∂h, h)

In [ ]:
plot(∂u, marker=:circle)
plot!(xᶜ[1:n], ∂udata_theoretical[1:n])

In [ ]:
plot(∂h, marker=:circle)
plot!(xᶠ[1:n], ∂hdata_theoretical[1:n])

Can we make it faster?

In [ ]:
@inline δᶜ!(δf, i, f) = @inbounds δf[i] = f[i+1] - f[i]
@inline δᶠ!(δf, i, f) = @inbounds δf[i] = f[i] - f[i-1]


function ∂!(∂f::Field{<:Face}, i, f::Field{<:Cell})
    δᶠ!(∂f.data, i, f.data)
    @inbounds ∂f.data[i] /= f.grid.dx
end

function ∂!(∂f::Field{<:Cell}, i, f::Field{<:Face})
    δᶜ!(∂f.data, i, f.data)
    @inbounds ∂f.data[i] /= f.grid.dx
end;

In [ ]:
function ∂_better!(∂f::Field, f::Field)
    for i in 1:f.grid.n
        @inbounds ∂!(∂f, i, f)
    end
end;

In [ ]:
∂_better!(∂u, u)
∂_better!(∂h, h)

In [ ]:
@btime ∂_better!(∂u, u)

In [ ]:
@btime ∂_better!(∂h, h)

In [ ]:
plot(∂u.grid.xᶜ[1:n], ∂u.data[1:n], marker=:circle)
plot!(xᶜ[1:n], ∂udata_theoretical[1:n])

In [ ]:
plot(∂h.grid.xᶜ[1:n], ∂h.data[1:n], marker=:circle)
plot!(xᶜ[1:n], ∂hdata_theoretical[1:n])